# CAPSTONE PROJECT WEEK 3

## 1. Installing required libraries

In [2]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge lxml --yes
print('Libraries installed!')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.4            |           py36_0          58 KB  conda-forge
    beautifulsoup4-4.8.1       |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.3 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.4-py36_0      conda-forge

The following packages will be UPDATED:

    beautiful

## 2. Importing the libraries

In [4]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json # library to handle JSON files

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

# Matplotlib and associated plotting modules
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import matplotlib.colors as colors

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler, normalize, scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score

print('Libraries imported!')

Libraries imported!


## 3. Scraping and retrieving data from the Wikipedia page into a Pandas DataFrame.

In [5]:
# Reading the Data

df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
print('Before dropping the rows, Dataframe Shape is:', df.shape)
df = df[df['Borough'] != 'Not assigned'] #Dropping the boolean values as mentioned
df = df.reset_index(drop=True) # Reseting the index 

# Changing 'not assigned'values in Neighborhoods Column
for i in df['Neighbourhood'].index:  
    if df['Neighbourhood'].loc[i] == 'Not assigned':
        df['Neighbourhood'].loc[i] = df['Borough'].loc[i]
    

df.head(20)


Before dropping the rows, Dataframe Shape is: (288, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## 4. Grouping Neighborhoods in the same Borough

In [12]:
# Grouping the dataframe for single postcode column as requested in Week3 Assignment

df_grouped = df.groupby(['Postcode','Borough'])['Neighbourhood'].agg([('Unique_Counts', 'nunique'), 
                                  ('Neighbourhood', lambda x: ', '.join(set(x)))])
df_grouped = df_grouped.drop(columns = 'Unique_Counts')
df_grouped.reset_index(inplace=True)
df_grouped.sort_values('Postcode')
df_grouped = df_grouped[['Postcode','Borough','Neighbourhood']]
df_grouped.columns = ['Postcode','Borough','Neighborhood']
df_grouped.head(20)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"
